In [1]:
!pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import torch
from transformers import ElectraTokenizer, ElectraForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from torch.optim import AdamW
from tqdm import tqdm
import torch.nn.functional as F
import os

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import ElectraTokenizer, ElectraForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from tqdm import tqdm
import numpy as np

# Data file path
file_path = '/content/drive/MyDrive/Kwargs/050. 기관 점수 예측 모델 (모델 B1)/final_dataset_test.csv'

# Read the data
data = pd.read_csv(file_path)

X = data.drop(columns=['MSCI_Score'])

# Y value is MSCI_SCORE
y = data['MSCI_Score']

#KoelectRA Talk Niser Road
tokenizer = ElectraTokenizer.from_pretrained('monologg/koelectra-base-v3-discriminator')

# Dataset class definition
class FeatureDataset(Dataset):
    def __init__(self, features, labels, tokenizer, max_len):
# Features processing
        if isinstance(features, pd.DataFrame):
            self.features = features.reset_index(drop=True)
        else:
            self.features = features  # 리스트나 다른 형식일 경우 그대로 사용

# Labels processing
        if isinstance(labels, (pd.Series, pd.DataFrame)):
            self.labels = labels.reset_index(drop=True)
        else:
            self.labels = labels  # 리스트나 다른 형식일 경우 그대로 사용

        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
# Text and numerical features from features
        if isinstance(self.features, pd.DataFrame):
            text = self.features.iloc[idx]['full_text']
            numeric_features = self.features.iloc[idx].drop(labels=['full_text', 'Company', 'date', 'Year']).astype(float).values
        else:
# Features is a list (for example, the dictionary of the list)
            text = self.features[idx]['full_text']
# 'FULL_TEXT', 'Company', 'Date', 'Yes'
            numeric_features = [float(value) for key, value in self.features[idx].items() if key not in ['full_text', 'Company', 'date', 'Year']]
            numeric_features = np.array(numeric_features)

# Extract label from labels
        if isinstance(self.labels, (pd.Series, pd.DataFrame)):
            label = self.labels.iloc[idx]
        else:
            label = self.labels[idx]

# Text tokenization (Clean_UP_TOKENIZATION_SPACES removal)
        encoding = self.tokenizer.encode_plus(
            text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'numeric_features': torch.tensor(numeric_features, dtype=torch.float),
            'labels': torch.tensor(label, dtype=torch.float)
        }

# Separation of learning set and verification set
train_features, val_features, train_labels, val_labels = train_test_split(X, y, test_size=0.2, random_state=42)

#Setting values ​​for hyper parameter exploration
max_len_values = [128, 256]
batch_size_values = [16, 32]
learning_rate_values = [2e-5, 3e-5, 5e-5]
num_epochs = 50

# Device setting
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Functions that convert tensors continuously
def make_contiguous(model):
    for param in model.parameters():
        param.data = param.data.contiguous()

# Training function definition
def train(model, train_dataloader, optimizer, device):
    model.train()
    total_loss = 0
    for batch in tqdm(train_dataloader, desc="Training"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        numeric_features = batch['numeric_features'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()

# Text output of the koelectra model
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits.squeeze()

# Text output and numerical feature combination
        combined_output = torch.cat((logits.unsqueeze(1), numeric_features), dim=1)

# Loss calculation (MSE Loss)
        loss = F.mse_loss(combined_output.sum(dim=1), labels)

        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_dataloader)

# Evaluation function definition
def evaluate(model, val_dataloader, device):
    model.eval()
    total_loss = 0
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for batch in tqdm(val_dataloader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            numeric_features = batch['numeric_features'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits.squeeze()

            combined_output = torch.cat((logits.unsqueeze(1), numeric_features), dim=1)
            loss = F.mse_loss(combined_output.sum(dim=1), labels)

            total_loss += loss.item()
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(combined_output.sum(dim=1).cpu().numpy())

    return total_loss / len(val_dataloader), all_labels, all_preds

# Hyper Parameter Navigation Loop
for max_len in max_len_values:
    for batch_size in batch_size_values:
        for learning_rate in learning_rate_values:
            print(f"Training with max_len={max_len}, batch_size={batch_size}, learning_rate={learning_rate}")

# Creation of dataset objects
            train_dataset = FeatureDataset(train_features, train_labels, tokenizer, max_len)
            val_dataset = FeatureDataset(val_features, val_labels, tokenizer, max_len)

#Dataloader
            train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

# Koelectra Model Road
            model = ElectraForSequenceClassification.from_pretrained('monologg/koelectra-base-v3-discriminator', num_labels=1)
            model.to(device)
            optimizer = AdamW(model.parameters(), lr=learning_rate)

# Variable for early termination
            best_val_loss = float('inf')
            patience = 5
            patience_counter = 0

# Training and evaluation
            for epoch in range(num_epochs):
                print(f"Epoch {epoch + 1}/{num_epochs}")
                train_loss = train(model, train_dataloader, optimizer, device)
                val_loss, val_labels_out, val_preds = evaluate(model, val_dataloader, device)
                print(f"Training Loss: {train_loss:.4f}")
                print(f"Validation Loss: {val_loss:.4f}")

                if val_loss < best_val_loss:
                    print(f"Validation loss improved from {best_val_loss:.4f} to {val_loss:.4f}. Saving model...")
                    best_val_loss = val_loss
                    patience_counter = 0

#Save the best model
                    best_model_dir = f"/content/drive/MyDrive/Kwargs/050. 기관 점수 예측 모델 (모델 B1)/best_model_len{max_len}_batch{batch_size}_lr{learning_rate}"
                    if not os.path.exists(best_model_dir):
                        os.makedirs(best_model_dir)
                    make_contiguous(model)
                    model.save_pretrained(best_model_dir)
                    tokenizer.save_pretrained(best_model_dir)
                    print(f"Best model and tokenizer saved to {best_model_dir}")
                else:
                    patience_counter += 1
                    print(f"Validation loss did not improve. Patience counter: {patience_counter}/{patience}")

                if patience_counter >= patience:
                    print("Early stopping triggered. Stopping training...")
                    break


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Training with max_len=128, batch_size=16, learning_rate=2e-05


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/50


Evaluating: 100%|██████████| 76/76 [00:10<00:00,  7.36it/s]


...

Evaluating: 100%|██████████| 76/76 [00:10<00:00,  7.31it/s]


Training Loss: 1.3392
Validation Loss: 2.4069
Validation loss did not improve. Patience counter: 2/5
Epoch 6/50


Evaluating: 100%|██████████| 76/76 [00:10<00:00,  7.26it/s]


Training Loss: 1.1364
Validation Loss: 2.3098
Validation loss improved from 2.3327 to 2.3098. Saving model...
Best model and tokenizer saved to /content/drive/MyDrive/Kwargs/050. 기관 점수 예측 모델 (모델 B1)/best_model_len128_batch16_lr2e-05
Epoch 7/50


Training:  86%|████████▋ | 262/303 [00:51<00:07,  5.27it/s]

In [ ]:
#Save model and torque nisor path
output_dir = "/content/drive/MyDrive/Kwargs/050. 기관 점수 예측 모델 (모델 B1)/"

#Model Road
model = ElectraForSequenceClassification.from_pretrained(output_dir)
model.eval()  # 평가 모드로 전환

#Talk Nizor Road
tokenizer = ElectraTokenizer.from_pretrained(output_dir)

# setting
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Read a new CSV file
new_file_path = '/content/drive/MyDrive/Kwargs/050. 기관 점수 예측 모델 (모델 B1)/sample_dataset_test.csv'
new_data = pd.read_csv(new_file_path)

# Data set class definition (the same as the class used in training)
class FeatureDataset(Dataset):
    def __init__(self, features, tokenizer, max_len):
        self.features = features
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
# Separate text and other numerical features
        text = self.features.iloc[idx]['full_text']

# Select only numerical features (select only numeric data)
        numeric_features = self.features.iloc[idx].drop(labels=['full_text', 'Company', 'date', 'Year']).astype(float).values

# Text tokenization
        encoding = self.tokenizer.encode_plus(
            text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

# Return
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'numeric_features': torch.tensor(numeric_features, dtype=torch.float),
        }

# Parameter setting
max_len = 128
batch_size = 16

# Creation of dataset objects
predict_dataset = FeatureDataset(new_data, tokenizer, max_len)

#Dataloader
predict_dataloader = DataLoader(predict_dataset, batch_size=batch_size, shuffle=False)

#Depation of predictive functions
def predict(model, dataloader, device):
    model.eval()
    predictions = []
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Predicting"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            numeric_features = batch['numeric_features'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits.squeeze()

            combined_output = torch.cat((logits.unsqueeze(1), numeric_features), dim=1)
            final_output = combined_output.sum(dim=1)  # 예: 간단히 합산하여 최종 출력 계산

            predictions.extend(final_output.cpu().numpy())

    return predictions

# Prediction performance
predictions = predict(model, predict_dataloader, device)

# Check for predictive value distribution
predicted_label_distribution = pd.Series(predictions).value_counts(bins=10)
print("Predicted label distribution:")
print(predicted_label_distribution)


Predicting: 100%|██████████| 441/441 [01:25<00:00,  5.13it/s]

Predicted label distribution:
(3.645, 4.828]      1533
(1.279, 2.462]      1502
(4.828, 6.011]      1348
(2.462, 3.645]       955
(0.0965, 1.279]      873
(6.011, 7.194]       509
(-1.086, 0.0965]     155
(7.194, 8.377]       148
(8.377, 9.56]         15
(-2.282, -1.086]      12
Name: count, dtype: int64


In [ ]:
# Convert the prediction result to Pandas Series
predictions_series = pd.Series(predictions)

# Confirmation check
missing_values = predictions_series.isnull().sum()
print(f"Number of missing values in predictions: {missing_values}")

Number of missing values in predictions: 0


In [ ]:
# Add the prediction result to the data frame
new_data['predicted_label'] = predictions

#Save data containing predictive results
output_file_path = '/content/drive/MyDrive/Kwargs/050. 기관 점수 예측 모델 (모델 B1)/결과/predictions.csv'
new_data.to_csv(output_file_path, index=False, encoding='utf-8-sig')

print(f"Predictions saved to {output_file_path}")


Predictions saved to /content/drive/MyDrive/Kwargs/모델B1/결과/predictions.csv


In [ ]:
# CSV file path
file_path = '/content/drive/MyDrive/Kwargs/050. 기관 점수 예측 모델 (모델 B1)/결과/predictions.csv'

# Read the data
data = pd.read_csv(file_path)

# Select only the necessary columns
data = data[['Year', 'Company', 'predicted_label']]

#Modter with Company and Yes, then the average calculation of predicted_label
grouped_data = data.groupby(['Company', 'Year']).agg(average_label=('predicted_label', 'mean')).reset_index()
print(grouped_data.to_string())

     Company  Year  average_label
0       KB금융  2019       5.016282
1       KB금융  2020       5.109671
2       KB금융  2021       5.996064
3       KB금융  2022       6.966534
4       KB금융  2023       7.012989
5      NAVER  2019       4.815658
6      NAVER  2020       4.797409
7      NAVER  2021       6.733145
8      NAVER  2022       6.642238
9      NAVER  2023       5.643522
10    SK하이닉스  2019       2.097431
11    SK하이닉스  2020       3.649572
12    SK하이닉스  2021       3.710508
13    SK하이닉스  2022       4.903346
14    SK하이닉스  2023       4.920477
15        기아  2019       1.796560
16        기아  2020       1.301239
17        기아  2021       2.058400
18        기아  2022       1.960288
19        기아  2023       2.774900
20     삼성SDI  2019       4.350137
21     삼성SDI  2020       4.419508
22     삼성SDI  2021       4.213435
23     삼성SDI  2022       4.739213
24     삼성SDI  2023       4.776360
25      삼성물산  2019       3.249006
26      삼성물산  2020       4.275316
27      삼성물산  2021       4.703798
28      삼성물산  

In [ ]:
#Save the result as a new CSV file
output_file_path = '/content/drive/MyDrive/Kwargs/050. 기관 점수 예측 모델 (모델 B1)/결과/average_predictions.csv'
grouped_data.to_csv(output_file_path, index=False)

print(f"결과가 {output_file_path}에 저장되었습니다.")

결과가 /content/drive/MyDrive/Kwargs/모델B1/결과/average_predictions.csv에 저장되었습니다.
